In [69]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torchvision import models

import pickle
from tqdm import tqdm, auto

import os

In [53]:
class AudioDataset(Dataset):
    def __init__(self, pkl_dir, transforms=None):
        self.data = []
        self.length = 1500
        self.transforms = transforms
        with open(pkl_dir, "rb") as f:
            self.data = pickle.load(f)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        entry = self.data[idx]
        output_data = {}
        values = entry["values"].reshape(-1, 128, self.length)
        values = torch.Tensor(values)
        if self.transforms:
            values = self.transforms(values)
        target = torch.LongTensor([entry["target"]])
        return (values, target)

    
def fetch_dataloader(pkl_dir, batch_size, num_workers):
    dataset = AudioDataset(pkl_dir)
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=num_workers)
    return dataloader

In [55]:
train_loader = fetch_dataloader( "data/training128mel1.pkl", 16, 8)
val_loader = fetch_dataloader("data/validation128mel1.pkl", 16, 8)

In [80]:
def train_epoch(model, device, data_loader, optimizer, loss_fn):
    model.train()
    loss_total, cnt = 0, 0

    with tqdm(total=len(data_loader)) as t:
        for batch_idx, data in enumerate(data_loader):
            inputs = data[0].to(device)
            target = data[1].squeeze(1).to(device)

            outputs = model(inputs)

            loss = loss_fn(outputs, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_total += loss.item()
            cnt += 1

            t.set_postfix(loss='{:05.3f}'.format(loss_total / cnt))
            t.update()
    return loss_total / cnt

def validate(model, device, test_loader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            inputs = data[0].to(device)
            target = data[1].squeeze(1).to(device)

            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return (100*correct/total)

In [76]:
class Inception(torch.nn.Module):
	def __init__(self, dataset, pretrained=True):
		super(Inception, self).__init__()
		num_classes = 50 if dataset=="ESC" else 10
		self.model = models.inception_v3(pretrained=pretrained, aux_logits=False)
		self.model.fc = torch.nn.Linear(2048, num_classes)

	def forward(self, x):
		output = self.model(x)
		return output

In [78]:
device = torch.device("cpu")

model = Inception("GTZAN", True).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)
loss_e1 = train_epoch(model, device, train_loader, optimizer, loss_fn)

In [82]:
acc_e1 = validate(model, device, val_loader)
print("Epoch 1 Loss:{} Valid Acc:{}".format(loss_e1, acc_e1))

Epoch 1 Loss:1.798953764438629 Valid Acc:67.5


## My netwurk.

In [36]:
class MyNetwurk(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(MyNetwurk, self).__init__()
        c, _, _ = input_size
        
        self.convlayers = torch.nn.Sequential(
            torch.nn.Conv2d(c, 6, (3, 3)),
            torch.nn.BatchNorm2d(6),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d((2, 2), stride=2), 
            
            torch.nn.Conv2d(6, 16, (3, 3)), 
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d((2, 2), stride=2),
            
            torch.nn.Conv2d(16, 64, (3, 3)), 
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d((2, 2), stride=2),
        )

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(256, 120),
            torch.nn.BatchNorm1d(120),
            torch.nn.ReLU(inplace=True),
            
            torch.nn.Linear(120, 60),
            torch.nn.BatchNorm1d(60),
            torch.nn.ReLU(inplace=True),

            torch.nn.Linear(60, num_classes),
        )

    def forward(self, x):
        x = self.convlayers(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [37]:
net = MyNetwurk((3, 1234, 1234), 10)

In [42]:
with open("test.pkl", 'wb') as f:
    pickle.dump(net, f)

In [43]:
with open("test.pkl", 'rb') as f:
    obj = pickle.load(f)

obj

MyNetwurk(
  (convlayers): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stat